# Vaccine Development with Dynamic Programming

As the CEO of a biotech company which is considering the development of a new vaccine. Starting at phase 0 (state 0), the drug develpment can stay in the same state or advance to "phase 1  with promising results" (state 1) or advance to "phase 1 with disappointing results" (state 2), or fail completely (state 4). At phase 1, the drug can stay in the same state, fail or become a success (state 3), in which case you will sell its patent to a big pharma company for \$10 million.
These state transitions happen from month to month, and at each state, you have the option to make an additional investment of \$100,000, which increases the chances of success.

After careful study, your analysts develop the program below to simulate different scenarios using statistical data from similar projects.

Use a discount factor of 0.996.

-  Write a policy iteration or value iteration code to compute the value of this project. Please print the full V (value function) vector.

In [ ]:
import numpy as np
class MDP():
  def __init__(self):
    self.A = [0, 1]
    self.S = [0, 1, 2, 3, 4]

    P0 = np.array([[0.5, .15, .15, 0, .20],
                   [0, .5, .0, .25, .25],
                   [0, 0, .15, .05, .8],
                   [0, 0, 0, 0, 1],
                   [0, 0, 0, 0, 1]])

    R0 = np.array([0, 0, 0, 10, 0])

    P1 = np.array([[0.5, .25, .15, 0, .10],
                   [0, .5, .0, .35, .15],
                   [0, 0, .20, .05, .75],
                   [0, 0, 0, 0, 1],
                   [0, 0, 0, 0, 1]])

    R1 = np.array([-0.1, -0.1, -0.1, 10, 0])

    self.P = [P0, P1]
    self.R = [R0, R1]

  def step(self, s, a):
    s_prime = np.random.choice(len(self.S), p=self.P[a][s])
    R = self.R[a][s]
    if s_prime == 4:
      done = True
    else:
      done = False
    return s_prime, R, done

  def simulate(self, s, a, π):
    done = False
    t = 0
    history = []
    while not done:
      if t > 0:
        a = π[s]
      s_prime, R, done = self.step(s, a)
      history.append((s, a, R))
      s = s_prime
      t += 1

    return history

You can access the transition probability matrices and the reward vector as follows:

In [ ]:
mdp = MDP()
P = mdp.P
R = mdp.R


s = 2 # current state
s_prime = 4  # next state
a = 1  # chosen action

# Probability of transition from state s (2) to s_prime (4) if action == a (1):
print(P[a][s, s_prime])

# Reward at state s if action = a
print(R[a][s])

In [ ]:
import numpy as np

class VaccineMDP:
    def __init__(self):
        # States: 0 (initial), 1 (promising phase 1), 2 (disappointing phase 1),
        # 3 (success), 4 (failure)
        self.states = [0, 1, 2, 3, 4]

        # Actions: 0 (no additional investment), 1 (invest $100,000)
        self.actions = [0, 1]

        # Discount factor
        self.gamma = 0.996

        # Transition probabilities without additional investment (P0)
        self.P0 = np.array([
            [0.5, 0.15, 0.15, 0, 0.20],  # state 0
            [0, 0.5, 0.0, 0.25, 0.25],   # state 1
            [0, 0, 0.15, 0.05, 0.80],    # state 2
            [0, 0, 0, 0, 1],             # state 3 (success)
            [0, 0, 0, 0, 1]              # state 4 (failure)
        ])

        # Transition probabilities with additional investment (P1)
        self.P1 = np.array([
            [0.5, 0.25, 0.15, 0, 0.10],  # state 0
            [0, 0.5, 0.0, 0.35, 0.15],   # state 1
            [0, 0, 0.20, 0.05, 0.75],    # state 2
            [0, 0, 0, 0, 1],             # state 3 (success)
            [0, 0, 0, 0, 1]              # state 4 (failure)
        ])

        # Rewards without additional investment (R0)
        self.R0 = np.array([0, 0, 0, 10, 0])

        # Rewards with additional investment (R1)
        # Includes the cost of investment (-0.1 = -$100,000)
        self.R1 = np.array([-0.1, -0.1, -0.1, 10, 0])

    def value_iteration(self, epsilon=1e-6, max_iterations=1000):
        """
        Perform value iteration to compute the optimal value function
        """
        # Initialize value function
        V = np.zeros(len(self.states))

        for _ in range(max_iterations):
            # Store previous value function to check convergence
            V_prev = V.copy()

            # Update value for each state
            for s in range(len(self.states)):
                # Compute value for both actions (no investment and investment)
                # If in terminal states (3 or 4), value is just the reward
                if s in [3, 4]:
                    V[s] = self.R0[s]
                    continue

                # Compute expected values for both actions
                action_values = []
                for a in self.actions:
                    # Choose appropriate transition and reward matrices
                    P = self.P0 if a == 0 else self.P1
                    R = self.R0 if a == 0 else self.R1

                    # Compute expected value
                    expected_value = R[s]
                    for s_next in range(len(self.states)):
                        expected_value += self.gamma * P[s, s_next] * V_prev[s_next]

                    action_values.append(expected_value)

                # Choose the action with maximum value
                V[s] = max(action_values)

            # Check for convergence
            if np.max(np.abs(V - V_prev)) < epsilon:
                break

        return V

# Run the value iteration
mdp = VaccineMDP()
optimal_values = mdp.value_iteration()

# Print the optimal value function
print("Optimal Value Function (V):")
for s, value in enumerate(optimal_values):
    print(f"State {s}: {value:.4f}")

Optimal Value Function (V):
State 0: 3.3207
State 1: 6.7450
State 2: 0.5855
State 3: 10.0000
State 4: 0.0000
